# 🚀 Crypto Scraper with **CoinCap** (no API‑Key)
Descarga hasta 2 000 activos, clasifica por narrativa (AI, Gaming, RWA, Meme) y genera OHLC 1 año.

## 0 · Instalar dependencias

In [1]:
!python -m pip install -q requests pandas tqdm aiohttp nest_asyncio pyarrow

## 1 · Parámetros y regex narrativas

In [4]:
import requests, json, time, tqdm, pandas as pd, re

BATCH  = 100
MAX_ASSETS = 2000
HEADERS = {
    "accept": "application/json",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
}

rows  = []
for offset in tqdm.tqdm(range(0, MAX_ASSETS, BATCH), desc="Tickers"):
    params = dict(limit=BATCH, offset=offset)
    for attempt in range(4):
        r = requests.get(
            "https://api.coincap.io/v2/assets",
            params=params, headers=HEADERS, timeout=30
        )
        if "application/json" not in r.headers.get("content-type", ""):
            time.sleep(1 + attempt)               # banner HTML → espera
            continue
        try:
            data = r.json()["data"]
            break                                 # JSON ok
        except json.JSONDecodeError:
            time.sleep(1 + attempt)
    else:
        print("⚠️  lote", offset, "saltado: respuesta no-JSON")
        continue
    rows.extend(data)
    if len(data) < BATCH:                         # fin lista
        break

print("Total bruto:", len(rows))


Tickers:   5%|▌         | 1/20 [00:12<03:57, 12.49s/it]

⚠️  lote 0 saltado: respuesta no-JSON


Tickers:  10%|█         | 2/20 [00:25<03:48, 12.69s/it]

⚠️  lote 100 saltado: respuesta no-JSON


Tickers:  15%|█▌        | 3/20 [00:37<03:28, 12.26s/it]

...

⚠️  lote 1700 saltado: respuesta no-JSON


Tickers:  95%|█████████▌| 19/20 [03:50<00:12, 12.25s/it]

⚠️  lote 1800 saltado: respuesta no-JSON


Tickers: 100%|██████████| 20/20 [04:03<00:00, 12.16s/it]

⚠️  lote 1900 saltado: respuesta no-JSON
Total bruto: 0


## 2 · Descargar listado de activos CoinCap

In [5]:
base_url = "https://api.coincap.io/v2/assets"
rows=[]
for offset in tqdm.tqdm(range(0, MAX_ASSETS, BATCH), desc="Tickers"):
    params=dict(limit=BATCH, offset=offset)
    data=requests.get(base_url, params=params, timeout=30).json()["data"]
    rows.extend(data)
    if len(data)<BATCH: break  # fin
print("Total filas:", len(rows))

df=pd.DataFrame(rows)
df["price"] = pd.to_numeric(df["priceUsd"], errors="coerce")
df["volume"] = pd.to_numeric(df["volumeUsd24Hr"], errors="coerce")
df["market_cap"] = pd.to_numeric(df["marketCapUsd"], errors="coerce")

df["narrative"]= (df["id"]+" "+df["symbol"]+" "+df["name"]).apply(tag)
df=df.dropna(subset=["narrative"]).reset_index(drop=True)

print("\nDistribución narrativa:")
print(df["narrative"].value_counts())
print("Tokens con narrativa:", len(df))

df[["id","symbol","name","narrative","price","volume","market_cap"]].to_csv(OUT_CSV,index=False)
print("📁 CSV guardado:", OUT_CSV)


Tickers:   0%|          | 0/20 [00:00<?, ?it/s]


JSONDecodeError: Extra data: line 1 column 5 (char 4)

## 3 · Descargar OHLC histórico 1 año (CoinCap)

In [ ]:
start_ms = int((datetime.datetime.utcnow() - datetime.timedelta(days=DAYS_OHLC)).timestamp()*1000)
end_ms   = int(datetime.datetime.utcnow().timestamp()*1000)

async def fetch_ohlc(sess, cid):
    url=f"https://api.coincap.io/v2/assets/{cid}/history"
    params=dict(interval="d1", start=start_ms, end=end_ms)
    async with sess.get(url, params=params, timeout=25) as r:
        if r.status!=200: return []
        data=(await r.json())["data"]
        return [{'id':cid,
                 'date':d['date'][:10],
                 'close':float(d['priceUsd'])} for d in data]

async def gather_ohlc(ids):
    sem=asyncio.Semaphore(CONC_OHLC)
    rows=[]
    async with aiohttp.ClientSession() as sess:
        async def worker(cid):
            async with sem:
                try: rows.extend(await fetch_ohlc(sess,cid))
                except: pass
        tasks=[worker(cid) for cid in ids]
        for _ in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="OHLC"):
            await _
    return rows

ids=df['id'].tolist()
t0=time.time()
ohlc_rows=asyncio.get_event_loop().run_until_complete(gather_ohlc(ids))
print(f"⏱️ OHLC descargado en {time.time()-t0:.1f}s — filas {len(ohlc_rows):,}")

df_ohlc=pd.DataFrame(ohlc_rows).set_index(['date','id']).sort_index()
df_ohlc.to_parquet(OUT_OHLC)
print("✅ OHLC guardado:", OUT_OHLC)
